In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.optimize import curve_fit
from lmfit import Model, minimize, Parameters, report_fit
from lmfit.models import GaussianModel, PolynomialModel
import astropy.units as u
import pandas as pd

In [2]:
def Gauss(x, h1, c1, w1):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2))
    return G

def doubleGauss(x, h1, c1, w1, h2, c2, w2):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2))
    return G

def tripleGauss(x, h1, c1,w1, h2, c2, w2, h3, c3, w3):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2)) + h3*np.exp(-(x-c3)**2/(2*w3**2))
    return G


In [3]:
def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf)):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe
    '''
    OH67 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/CO' + str(l) + 'p' + str(b) + '.txt')



    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)', 
                                        'CenterVel (km/s)',
                                       'FWHM (km/s)',
                                       ])

    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']

    #fit HI data
    if n_HI == 1:

        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [parametersHI[2]]

        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        fitHIc[i],
                                                        fitHIw[i],
                                                        ]

        
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [parametersHI[2], parametersHI[5]]
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        fitHIc[i],
                                                        fitHIw[i],
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [parametersHI[2], parametersHI[5], parametersHI[8]]

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        fitHIc[i],
                                                        fitHIw[i],
                                                        ]

            
    #fit 65 data
    
    if n_65 == 1:
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [parameters65[2]]

        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        fit65c[i],
                                                        fit65w[i],
                                                        ]
    
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [parameters65[2], parameters65[5]]

        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        fit65c[i],
                                                        fit65w[i],
                                                        ]

            
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [parameters65[2], parameters65[5], parameters65[8]]

        
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        fit65c[i],
                                                        fit65w[i],
                                                        ]


    #fit 67 data
    
    if n_67 == 1:
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [parameters67[2]]

        
        
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        fit67c[i],
                                                        fit67w[i],
                                                        ]

        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [parameters67[2], parameters67[5]]

                
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        fit67c[i],
                                                        fit67w[i],
                                                        ]
            

            
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [parameters67[2], parameters67[5], parameters67[8]]

                
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        fit67c[i],
                                                        fit67w[i],
                                                        ]

            
    #fit CO data
    
    if n_CO == 1:
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [parametersCO[2]]

        
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        fitCOc[i],
                                                        fitCOw[i],
                                                        ]

        
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [parametersCO[2], parametersCO[5]] 

    
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        fitCOc[i],
                                                        fitCOw[i],
                                                        ]

    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [parametersCO[2], parametersCO[5], parametersCO[8]] 

        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        fitCOc[i],
                                                        fitCOw[i],
                                                        ]


               
    resultsTable = resultsTable.drop(0)
    return resultsTable
    #df = fit_coord(...)
    #df.save_csv()

    #files = glob.glob(*.csv)
    #for file in files:
        #df.read_csv(...)
        #df_full.concatenate(df)
        #df_full.summary()

In [4]:
fit_coord(119070, 2800)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,119070,2800,HI,1,28.026152,-4.774549,2.794639
2,119070,2800,65,1,0.043520,-4.251790,1.081164
3,119070,2800,67,1,0.086602,-4.358241,1.103239
4,119070,2800,CO,1,1.613065,-3.779779,0.865799


In [5]:
fit_coord(119210, 2000)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,119210,2000,HI,1,25.963366,-7.092726,8.563325
2,119210,2000,65,1,0.015542,0.695469,1.411649
3,119210,2000,67,1,0.035690,0.156634,1.480291
4,119210,2000,CO,1,0.640718,0.650001,0.732573


In [9]:
fit_coord(120500, 1863)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,120500,1863,HI,1,27.612392,-5.419586,7.871846
2,120500,1863,65,1,0.056665,-0.806004,0.882485
3,120500,1863,67,1,0.098978,-0.888053,0.991228
4,120500,1863,CO,1,2.971228,-0.813725,0.690755


In [10]:
fit_coord(120500, 2963)

D:\python stuff\conda\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,120500,2963,HI,1,22.742624,-4.095128,4.735093
2,120500,2963,65,1,0.035391,-6.186348,-0.273821
3,120500,2963,67,1,0.022943,-5.066210,2.599434
4,120500,2963,CO,1,0.046671,-0.001451,0.014660


In [11]:
fit_coord(121070, 2175)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,121070,2175,HI,1,15.235957,-5.053578,7.037631
2,121070,2175,65,1,0.077346,-5.794758,-0.676482
3,121070,2175,67,1,0.123891,-5.836851,0.887564
4,121070,2175,CO,1,3.393214,-5.278035,1.757821


In [12]:
fit_coord(121930, 1975)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,121930,1975,HI,1,19.581054,-7.167073,9.470173
2,121930,1975,65,1,0.072349,1.297824,-0.239236
3,121930,1975,67,1,0.010129,0.104736,2.345581
4,121930,1975,CO,1,0.859033,1.332860,0.569361


In [13]:
fit_coord(122220, 1875)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,122220,1875,HI,1,23.153382,-8.302844,9.323011
2,122220,1875,65,1,0.013580,1.981341,1.154046
3,122220,1875,67,1,0.027175,1.999606,0.803392
4,122220,1875,CO,1,0.154372,-2.229722,7.251700


In [15]:
fit_coord(122500, 2900)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,122500,2900,HI,1,10.713089,-9.966956,11.996990
2,122500,2900,65,1,0.012392,-6.669039,1.331156
3,122500,2900,67,1,0.010909,-5.722605,3.122121
4,122500,2900,CO,1,-0.633424,-1.259937,0.168426


In [16]:
fit_coord(123360, 2075)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,123360,2075,HI,1,21.803903,-5.073099,9.849233
2,123360,2075,65,1,0.020387,1.480316,2.777186
3,123360,2075,67,1,0.043099,1.913554,2.243140
4,123360,2075,CO,1,1.376642,0.727804,2.274045


In [17]:
fit_coord(123500, 1963)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,123500,1963,HI,1,21.803903,-5.073099,9.849233
2,123500,1963,65,1,0.021746,-3.947261,1.690185
3,123500,1963,67,1,0.034349,-3.964272,1.485942
4,123500,1963,CO,1,0.943788,0.679348,-0.188614


In [18]:
fit_coord(125070, 1950)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,125070,1950,HI,1,17.913349,-5.532131,11.126012
2,125070,1950,65,1,0.059559,0.686141,0.555600
3,125070,1950,67,1,0.106404,0.660790,0.620396
4,125070,1950,CO,1,1.962675,0.680557,0.601743


In [19]:
fit_coord(125220, 3250)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,125220,3250,HI,1,36.662619,1.152745,2.775750
2,125220,3250,65,1,0.068332,0.234129,0.961648
3,125220,3250,67,1,0.120393,0.069041,-1.048728
4,125220,3250,CO,1,0.863866,0.389683,0.993339


In [6]:
fit_coord(127500, 2088)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,127500,2088,HI,1,21.886240,-2.254613,9.868741
2,127500,2088,65,1,0.025575,-1.365188,2.235869
3,127500,2088,67,1,0.052604,-1.028931,2.015442
4,127500,2088,CO,1,2.598015,-0.610891,1.288129


In [7]:
fit_coord(137160, 4300)

D:\python stuff\conda\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,137160,4300,HI,1,1.194838e+01,7.550833e+00,2.510706
2,137160,4300,65,1,3.101224e-03,-1.008849e+00,1.629755
3,137160,4300,67,1,-3.248013e-04,-8.616023e-01,0.071983
4,137160,4300,CO,1,-1.617513e-19,1.130883e-09,1.000000


In [8]:
fit_coord(142210, 2325)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,142210,2325,HI,1,25.508466,-2.111719,5.592580
2,142210,2325,65,1,0.043649,-1.955868,0.789529
3,142210,2325,67,1,0.084863,-2.042813,0.820334
4,142210,2325,CO,1,2.504879,-2.169310,0.711052


In [9]:
fit_coord(143000, 3850)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,143000,3850,HI,1,25.340131,3.250074,3.144595
2,143000,3850,65,1,0.053961,2.292664,0.921137
3,143000,3850,67,1,0.097741,2.225671,0.918526
4,143000,3850,CO,1,2.491926,2.302071,0.795076


In [10]:
fit_coord(146070, 1775)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,146070,1775,HI,1,31.705536,-0.341993,4.118570
2,146070,1775,65,1,0.169170,0.591286,-0.687075
3,146070,1775,67,1,0.230267,0.487612,-0.723211
4,146070,1775,CO,1,4.350812,0.258126,0.899220


In [11]:
fit_coord(146200, 3963)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,146200,3963,HI,1,10.893104,2.048238,5.877930
2,146200,3963,65,1,0.017295,0.399105,0.695627
3,146200,3963,67,1,0.022717,-0.098259,1.241909
4,146200,3963,CO,1,1.779890,0.001515,0.816434


In [12]:
fit_coord(146930, 2075)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,146930,2075,HI,1,37.110928,-1.210916,3.014247
2,146930,2075,65,1,0.033501,0.136408,0.642267
3,146930,2075,67,1,0.057882,-0.134523,0.841744
4,146930,2075,CO,1,0.697665,0.055885,0.695648


In [13]:
fit_coord(147000, 4038)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,147000,4038,HI,1,11.355956,0.643146,4.992655
2,147000,4038,65,1,0.018666,-1.050442,1.725202
3,147000,4038,67,1,0.040348,-1.812333,1.042366
4,147000,4038,CO,1,1.510091,-1.845083,0.883508


In [14]:
fit_coord(147200, 4075)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,147200,4075,HI,1,19.663554,1.742909,3.782216
2,147200,4075,65,1,0.054946,3.812875,0.683664
3,147200,4075,67,1,0.076702,3.683304,0.731789
4,147200,4075,CO,1,4.015911,3.726927,-0.644383


In [15]:
fit_coord(148200, 3863)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,148200,3863,HI,1,13.374745,0.279846,4.602784
2,148200,3863,65,1,0.041634,-2.200168,0.860289
3,148200,3863,67,1,0.047720,-1.827805,1.207130
4,148200,3863,CO,1,1.576454,-2.316782,0.693315


In [16]:
fit_coord(148210, 2350)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,148210,2350,HI,1,15.359772,-0.577543,7.212826
2,148210,2350,65,1,0.044365,1.283639,0.609142
3,148210,2350,67,1,0.066329,1.370475,-0.671339
4,148210,2350,CO,1,3.165647,1.304324,0.545670


In [17]:
fit_coord(150120, 4067)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,150120,4067,HI,1,1.060897e+01,1.221581e+00,5.192746
2,150120,4067,65,1,2.465818e-03,1.338215e+00,5.567353
3,150120,4067,67,1,3.396686e-03,-3.658921e+00,3.183563
4,150120,4067,CO,1,-1.617513e-19,1.130883e-09,1.000000


In [18]:
fit_coord(151500, 1925)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,151500,1925,HI,1,27.387199,-0.573307,4.941640
2,151500,1925,65,1,0.011849,-1.449924,0.812572
3,151500,1925,67,1,0.022660,-1.235308,1.298628
4,151500,1925,CO,1,1.513998,-1.399332,0.702123


In [19]:
fit_coord(156540, 3513)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,156540,3513,HI,1,15.205414,-0.789128,3.658962
2,156540,3513,65,1,0.021783,-2.966400,0.888325
3,156540,3513,67,1,0.031306,-3.043428,0.931788
4,156540,3513,CO,1,1.725985,-3.363716,-0.687419


In [20]:
fit_coord(156640, 3250)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,156640,3250,HI,1,23.456820,-0.742946,2.861469
2,156640,3250,65,1,0.002425,0.949288,3.763955
3,156640,3250,67,1,0.001190,-0.204544,-2.028822
4,156640,3250,CO,1,1.742431,-0.815809,1.018795


In [21]:
fit_coord(157350, 2200)

,l (deg),b (deg),line,comp,T_peak (K),CenterVel (km/s),FWHM (km/s)
1,157350,2200,HI,1,8.795683e+00,-1.574190e+00,12.684521
2,157350,2200,65,1,3.419824e-03,1.046830e+01,7.030360
3,157350,2200,67,1,-2.531804e-03,2.580562e+00,1.490589
4,157350,2200,CO,1,-1.617513e-19,1.130883e-09,1.000000
